In [43]:

from sklearn import *

import numpy as np
import pandas as pd
import sklearn as sk 
import sklearn.model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

df = pd.read_csv("./train.csv")

#extract the review summary columm
# print(df["summary"], df["overall"])

#array of all the summaries
summaries = df["summary"]

#array of the overalls
overall = df["overall"]

#array of product nums
products = df["amazon-id"]



0                                                 Beautiful
1                                Tranquillity In Motion !!!
2                                               Great Stuff
3                                                 Beautiful
4                                             tai chi music
                                ...                        
111093                                           Four Stars
111094    I'd you are looking for the music that they sa...
111095       FANTASTIC FILM! Loved this movie and the music
111096                          A great new CD with uptempo
111097                          Matt Marshak does it again!
Name: summary, Length: 111098, dtype: object 0         4
1         5
2         5
3         5
4         5
         ..
111093    4
111094    5
111095    5
111096    5
111097    5
Name: overall, Length: 111098, dtype: int64


In [2]:
#extracting IMDB movie reviews
moviesdf = pd.read_csv("./labeledTrainData.tsv", delimiter = "\t")
moviesdf.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
reviews = moviesdf["review"]
sentiment = moviesdf["sentiment"]

#create train test split
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(reviews, sentiment, random_state=1)


In [4]:
#create model
vect = CountVectorizer()


x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

#use logistic regression first
clf = LogisticRegression()
clf.fit(x_train, y_train)

/Users/gokulsrin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
#testing acc 
print(clf.score(x_test, y_test))

0.88


In [26]:
#using this to classify shit
data = vect.transform(summaries.values.astype('U'))
predictions = clf.predict(data)

wonderful 1


In [38]:
#accuracy analysis
total = len(summaries)
t = 0
for i in range(total):
    if (overall[i] >= 4 and predictions[i] == 1) or (overall[i] < 4 and predictions[i] == 0):
        t+=1
print(t/total)

0.7460980395686692


In [48]:
# get the average score for each product
dic = {} # product-id: array of scores
for i in range(len(products)):
    if dic.get(products[i]) == None:
        dic[products[i]] = [overall[i]]
    else:
        dic.get(products[i]).append(overall[i])
        
#transform dic into average scores, and then into 1 or 0 (corresponding to >4.5 and <4.5)
for product in dic.keys():
    s = sum(dic.get(product))
    s/= len(dic.get(product))
    if s >4.5:
        dic[product] = 1
    else:
        dic[product] = 0
    
print(dic)

{1877521326299865484: 0, 2828769427501176858: 1, 1609491003013328661: 1, 1683355681609577463: 0, 245546254853962888: 1, -7423206632058739702: 1, -2932232764923751785: 1, -6549800316484850226: 1, -8564211091894325720: 0, -4253147973590114159: 0, -4342138265689396389: 1, 7583542239035197041: 1, -475005613570216251: 1, -8971039467609925228: 0, 1418255743118192763: 0, -6565336230712636570: 1, 8728178293825196078: 1, 668084135650245809: 1, -2314511033920056425: 1, -7485037166509031190: 1, 260543842062698168: 0, 2039740548338929642: 1, -6627342038240248423: 0, 6564614632242065555: 1, -1408101816358885169: 0, -6908255106283322318: 0, 6028818170909227811: 0, 6932547228640535934: 1, 955485083203295824: 1, 1705492661016018024: 1, 9212043675364570807: 1, 6189278655559636060: 0, -5335891973127508414: 1, 502234485348765728: 0, -4803069149954555901: 1, -4166127553553467154: 0, 6733237573499936258: 0, -8028687232885424453: 0, 7045284943612035363: 1, -8216616717238534331: 1, -5002294654783500661: 0, -

In [54]:
#generate the average predicted sentiment for each product

# get the average sentiment for each product
dic2 = {} # product-id: array of sentiment
for i in range(len(products)):
    if dic2.get(products[i]) == None:
        dic2[products[i]] = [predictions[i]]
    else:
        dic2.get(products[i]).append(predictions[i])

#average out the sentiment for each project
for product in dic2.keys():
    s = sum(dic2.get(product))
    s/= len(dic2.get(product))
    dic2[product] = s
    
print(dic2)

{1877521326299865484: 0.6, 2828769427501176858: 1.0, 1609491003013328661: 1.0, 1683355681609577463: 1.0, 245546254853962888: 0.75, -7423206632058739702: 0.9166666666666666, -2932232764923751785: 0.7986486486486486, -6549800316484850226: 0.7647058823529411, -8564211091894325720: 0.0, -4253147973590114159: 0.38461538461538464, -4342138265689396389: 0.4, 7583542239035197041: 1.0, -475005613570216251: 0.75, -8971039467609925228: 1.0, 1418255743118192763: 1.0, -6565336230712636570: 1.0, 8728178293825196078: 0.75, 668084135650245809: 0.0, -2314511033920056425: 0.6666666666666666, -7485037166509031190: 1.0, 260543842062698168: 1.0, 2039740548338929642: 1.0, -6627342038240248423: 0.625, 6564614632242065555: 1.0, -1408101816358885169: 1.0, -6908255106283322318: 0.8, 6028818170909227811: 0.0, 6932547228640535934: 0.5, 955485083203295824: 0.6666666666666666, 1705492661016018024: 0.75, 9212043675364570807: 0.8333333333333334, 6189278655559636060: 0.5, -5335891973127508414: 1.0, 502234485348765728:

In [73]:
# we are creating [ [], [], ..., []]
#this is why we don't need to transform the data below
d2 = []
for i in dic2.keys():
    d2.append([dic2.get(i)])
d = []
for i in dic.keys():
    d.append([dic.get(i)])

In [74]:
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(d2, d, random_state=1)


#train model
clf2 = LogisticRegression()
clf2.fit(X_train, Y_train)

/Users/gokulsrin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gokulsrin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
#test model 
print(clf2.score(X_test,Y_test))

0.5952200303490136
